In [1]:
import sounddevice as sd


from source.processing import *
from config.settings import PROCESSING

from source.training import get_model

from utils.plotting import plot_spectrogram

2022-05-17 16:00:02.297021: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-17 16:00:02.297043: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
freq = fft_frequencies(sr=PROCESSING.sampling_rate, n_fft=PROCESSING.n_fft)
log_freq = log_scale_freq(freq, log_base=PROCESSING.log_base, num_samples=PROCESSING.num_samples)[0]

In [3]:
x_train, y_train, x_test, y_test, scalars = load_split_tensors(PROCESSING.tensors_folder)

In [51]:
batch = np.load('data/predictions/tensors/pred_x.npy')

In [52]:
restored_tensor = batch_to_tensor(batch)

In [53]:
restored_sg = tensor_to_spectrogram(restored_tensor, scalars['y'])

In [54]:
rescaled_restored_sg = inverse_log_scale(log_freq, restored_sg)

In [55]:
restored_audio = spectrogram_to_wav(rescaled_restored_sg, PROCESSING.sampling_rate)(win_length=PROCESSING.win_length, n_fft=PROCESSING.n_fft)

In [56]:
sd.play(*restored_audio)

In [57]:
sd.stop()

In [20]:
original = spectrogram_to_wav(inverse_log_scale(log_freq, tensor_to_spectrogram(batch_to_tensor(y_train[10:]), scalar=scalars['y'])))()

In [34]:
sd.play(*original)

In [35]:
sd.stop()